In [3]:
%run 3.inference.ipynb

Aresta [-55, 2, 17, 0.3] -> Classe 1 (Bom) (confiança 0.99)
Aresta [-85, 3.8, 30, 0.1] -> Classe 1 (Bom) (confiança 0.53)
Aresta [-35, 1.2, 5, 0.05] -> Classe 1 (Bom) (confiança 1.00)
Aresta [-88, 9.5, 95, 0.9] -> Classe 0 (Ruim) (confiança 1.00)
Aresta [-40, 8.2, 85, 0.8] -> Classe 0 (Ruim) (confiança 1.00)
Aresta [-87, 1.5, 8, 0.15] -> Classe 1 (Bom) (confiança 0.98)
Aresta [-62, 2.8, 32, 0.28] -> Classe 1 (Bom) (confiança 0.96)
Aresta [-58, 3.2, 28, 0.32] -> Classe 1 (Bom) (confiança 0.96)
Aresta [-52, 2.1, 20, 0.25] -> Classe 1 (Bom) (confiança 0.99)
Aresta [-48, 2.5, 25, 0.22] -> Classe 1 (Bom) (confiança 0.99)
Aresta [-50, 1.8, 22, 0.28] -> Classe 1 (Bom) (confiança 0.99)
Aresta [-75, 5.2, 65, 0.6] -> Classe 0 (Ruim) (confiança 1.00)
Aresta [-78, 4.8, 70, 0.65] -> Classe 0 (Ruim) (confiança 1.00)
Aresta [-72, 5.5, 60, 0.58] -> Classe 0 (Ruim) (confiança 1.00)
Aresta [-45, 4.5, 15, 0.7] -> Classe 1 (Bom) (confiança 0.56)
Aresta [-70, 1.9, 45, 0.2] -> Classe 1 (Bom) (confiança 0.94

In [ ]:
import numpy as np

# ====== Auxiliares ======
def clip01(x): 
    return max(0.0, min(1.0, float(x)))

# ====== Probabilidade do MLP (classe "Bom") ======
prob_ml = probs[:, 1].numpy()

# ====== Métricas Radnet ======
# Por aresta (1 valor por amostra)
n = len(test_data)

# Exemplos determinísticos e suaves só para mock
APMR_edge = np.clip(0.65 + 0.20*np.sin(np.linspace(0, 3*np.pi, n)), 0, 1)   # match de prefixo por par
MFSR_edge = np.clip(0.70 + 0.15*np.cos(np.linspace(0, 2*np.pi, n)), 0, 1)   # sucesso de encaminhamento por par

# Global (um único valor para todas as amostras)
IGD_GLOBAL = 0.62  # densidade de interesse da rede (ajuste conforme seu cenário)
IGD_vec = np.full(n, clip01(IGD_GLOBAL))

# ====== Fusão espacial (ponderada) ======
w_ml, w_ap, w_mf, w_ig = 0.50, 0.20, 0.15, 0.15
final_score = (w_ml*prob_ml +
               w_ap*APMR_edge +
               w_mf*MFSR_edge +
               w_ig*IGD_vec)

# ====== Decisão final ======
tau = 0.5
final_class = (final_score >= tau).astype(int)
CLASS_NAMES = {0: "Ruim", 1: "Bom"}

# ====== Resultados ======
print("\nSaída após fusão espacial (MLP + APMR_edge + MFSR_edge + IGD_GLOBAL):")
for i, row in enumerate(test_data):
    print(
        f"Aresta {row} | "
        f"MLP={prob_ml[i]:.2f}, APMR_edge={APMR_edge[i]:.2f}, MFSR_edge={MFSR_edge[i]:.2f}, IGD={IGD_vec[i]:.2f} "
        f"-> Score={final_score[i]:.2f} => Classe {final_class[i]} ({CLASS_NAMES[final_class[i]]})"
    )

fusion_scores = final_score.tolist()



Saída após fusão espacial (MLP + APMR_edge + MFSR_edge + IGD_GLOBAL):
Aresta [-55, 2, 17, 0.3] | MLP=0.99, APMR_edge=0.65, MFSR_edge=0.85, IGD=0.62 -> Score=0.85 => Classe 1 (Bom)
Aresta [-85, 3.8, 30, 0.1] | MLP=0.53, APMR_edge=0.68, MFSR_edge=0.85, IGD=0.62 -> Score=0.62 => Classe 1 (Bom)
Aresta [-35, 1.2, 5, 0.05] | MLP=1.00, APMR_edge=0.70, MFSR_edge=0.85, IGD=0.62 -> Score=0.86 => Classe 1 (Bom)
Aresta [-88, 9.5, 95, 0.9] | MLP=0.00, APMR_edge=0.73, MFSR_edge=0.84, IGD=0.62 -> Score=0.37 => Classe 0 (Ruim)
Aresta [-40, 8.2, 85, 0.8] | MLP=0.00, APMR_edge=0.75, MFSR_edge=0.84, IGD=0.62 -> Score=0.37 => Classe 0 (Ruim)
Aresta [-87, 1.5, 8, 0.15] | MLP=0.98, APMR_edge=0.78, MFSR_edge=0.83, IGD=0.62 -> Score=0.87 => Classe 1 (Bom)
Aresta [-62, 2.8, 32, 0.28] | MLP=0.96, APMR_edge=0.80, MFSR_edge=0.83, IGD=0.62 -> Score=0.86 => Classe 1 (Bom)
Aresta [-58, 3.2, 28, 0.32] | MLP=0.96, APMR_edge=0.81, MFSR_edge=0.82, IGD=0.62 -> Score=0.86 => Classe 1 (Bom)
Aresta [-52, 2.1, 20, 0.25] | M